In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
from scipy.stats import spearmanr

In [3]:
# Define the folder path where the CSV files are located
file_path = "/Users/niklaskiermeyer/Desktop/Codespace/DruxAI/results/slurm/big_model_results.csv"

results_val = pd.read_csv(file_path)

In [25]:
results_val.head()

,ground_truth,prediction,cells,drugs
0,[-0.14991311728954315],[-0.12373024225234985],ACH-000013,1-NAPHTHYL-PP1
1,[0.429168701171875],[-0.08158686757087708],ACH-000974,CP-724714
2,[-0.3943784832954407],[-0.25059542059898376],ACH-000915,OBATOCLAX
3,[0.2287609577178955],[0.12117030471563339],ACH-000312,RUCINOL
4,[0.32743167877197266],[0.005660128779709339],ACH-000351,VISMODEGIB


In [6]:
from torchmetrics import SpearmanCorrCoef  # Assuming you have implemented this metric
import torch
metric_train_rscore = SpearmanCorrCoef().to("cpu")
metric_train_rscore.reset()

/Users/niklaskiermeyer/anaconda3/envs/DruxAI/lib/python3.9/site-packages/torchmetrics/utilities/prints.py:43: UserWarning: Metric `SpearmanCorrcoef` will save all targets and predictions in the buffer. For large datasets, this may lead to large memory footprint.
  warnings.warn(*args, **kwargs)  # noqa: B028


In [7]:
prediction = results_val["prediction"].str.strip("[]").astype(float).to_list()
ground_truth = results_val["ground_truth"].str.strip("[]").astype(float).to_list()

In [8]:
metric_train_rscore(torch.tensor(prediction), torch.tensor(ground_truth))

tensor(0.7718)

In [9]:
r_value_S, _ = spearmanr(prediction, ground_truth)
print(f"Mean R Score: {r_value_S}")

Mean R Score: 0.7718161715097842


In [10]:
# Overall R-Score
r_value_SS, _ = spearmanr(results_val["prediction"], results_val["ground_truth"])
print(f"Mean R Score: {r_value_SS}")

Mean R Score: 0.39321731274370103


In [24]:
results_val.head()

,ground_truth,prediction,cells,drugs
0,[-0.14991311728954315],[-0.12373024225234985],ACH-000013,1-NAPHTHYL-PP1
1,[0.429168701171875],[-0.08158686757087708],ACH-000974,CP-724714
2,[-0.3943784832954407],[-0.25059542059898376],ACH-000915,OBATOCLAX
3,[0.2287609577178955],[0.12117030471563339],ACH-000312,RUCINOL
4,[0.32743167877197266],[0.005660128779709339],ACH-000351,VISMODEGIB


In [16]:
import pandas as pd
from scipy.stats import spearmanr

# Create DataFrame with variables within brackets
data_brackets = {
    "ground_truth": [[-0.1499], [0.4291], [-0.3944], [0.2288], [0.3274]],
    "prediction": [[-0.1237], [-0.0816], [-0.2506], [0.1212], [0.0057]],
    "cells": ["ACH-000013", "ACH-000974", "ACH-000915", "ACH-000312", "ACH-000351"],
    "drugs": ["1-NAPHTHYL-PP1", "CP-724714", "OBATOCLAX", "RUCINOL", "VISM"]
}

df_brackets = pd.DataFrame(data_brackets)

# Convert lists to single values
df_brackets["ground_truth"] = df_brackets["ground_truth"].apply(lambda x: x[0])
df_brackets["prediction"] = df_brackets["prediction"].apply(lambda x: x[0])

# Calculate Spearman correlation coefficient for DataFrame with variables within brackets
spearman_brackets, _ = spearmanr(df_brackets["ground_truth"], df_brackets["prediction"])

# Create DataFrame with variables not within brackets
data_no_brackets = {
    "ground_truth": [-0.1499, 0.4291, -0.3944, 0.2288, 0.3274],
    "prediction": [-0.1237, -0.0816, -0.2506, 0.1212, 0.0057],
    "cells": ["ACH-000013", "ACH-000974", "ACH-000915", "ACH-000312", "ACH-000351"],
    "drugs": ["1-NAPHTHYL-PP1", "CP-724714", "OBATOCLAX", "RUCINOL", "VISM"]
}

df_no_brackets = pd.DataFrame(data_no_brackets)

# Calculate Spearman correlation coefficient for DataFrame with variables not within brackets
spearman_no_brackets, _ = spearmanr(df_no_brackets["ground_truth"], df_no_brackets["prediction"])

print("Spearman correlation coefficient for DataFrame with variables within brackets:", spearman_brackets)
print("Spearman correlation coefficient for DataFrame with variables not within brackets:", spearman_no_brackets)


Spearman correlation coefficient for DataFrame with variables within brackets: 0.6
Spearman correlation coefficient for DataFrame with variables not within brackets: 0.6


In [ ]:
# Per Drug R score
entire_dataset_rscores = []  # Initialize a DataFrame to store the R scores for the entire dataset

r_scores = []  # Initialize a list to store R scores for the current fold
for group_name, group_data in results_val.groupby("drugs"):
    r_value, _ = spearmanr(group_data["prediction"], group_data["ground_truth"])
    r_scores.append({"Group": group_name, "Mean Drug R Score": r_value})
entire_dataset_rscores.extend(r_scores)
fold_mean_r_score = pd.DataFrame(r_scores)["Mean Drug R Score"].mean()
print(f"Mean R Score per drug: {fold_mean_r_score}")

entire_dataset_rscores = pd.DataFrame(entire_dataset_rscores).groupby("Group").mean().sort_values("Mean Drug R Score"
                                                                                                , ascending=False)
print(" \n Overall per drug mean R Score:", entire_dataset_rscores["Mean Drug R Score"].mean())

# Evaluation with Model Loaded weights

In [ ]:
# Checkpoint path
# Always safe specific config with a run! Important especially if seed is changed because then we have other splits!

ckpt_path = "/Users/niklaskiermeyer/Desktop/Codespace/DruxAI/results/training/ckpt_sgd_baseline.pt"

checkpoint = torch.load(ckpt_path)
cfg = checkpoint["config"]